# Data Collection 

### API and WEBSCRAPE articles

These articles will be used to:

* TRAIN and TEST MODEL 
    * from CNN, FOX, BREITBERT, and WASHINGTON POST
* CLASSIFY Stakeholder's Data using trained MODEL 
    * from THE HILL


In [ ]:
#! pip install pyLDavis

import pyLDAvis

pyLDAvis.enable_notebook()
from tqdm.auto import tqdm

import pyLDAvis.lda_model
import pyLDAvis.gensim_models

import numpy as np
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt
import os
import shutil
import re
import logging
import time
import zipfile
import requests
from bs4 import BeautifulSoup
import datetime
import re
import regex as rex
from collections import defaultdict, Counter
import random
import requests
from bs4 import BeautifulSoup
import datetime
import json
from wordcloud import WordCloud 
from tabulate import tabulate
from sklearn.svm import SVC


import sqlite3
import nltk
from string import punctuation
from nltk.corpus import stopwords
import re
import emoji
from nltk.metrics import ConfusionMatrix
import itertools
import collections

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
## Depracated:
# from sklearn.metrics import plot_confusion_matrix
## New version:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.decomposition import LatentDirichletAllocation

#import mysql.connector

# Set pandas global options
pd.options.display.max_rows = 17

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Connect to NewsAPI client

In [ ]:
from newsapi import NewsApiClient

api_key = os.environ['NewsAPIKey']

# Init
newsapi = NewsApiClient(api_key=api_key)

## Pull article URLs and info from API

### /v2/top-headlines/sources
sources = newsapi.get_sources()
print(sources)

In [ ]:
def news_api_urls(q=None,
                  s=None,
                  d_from='2023-05-01',
                  d_to='2023-05-31',
                  api_lst=[]):
    all_articles = newsapi.get_everything(q=q,
                                          sources=s,
                                          from_param=d_from,
                                          to=d_to,
                                          language='en',
                                          sort_by='relevancy',
                                          page=1)

    print(type(all_articles))
    print(all_articles)
    #print('Article list: ', all_articles['articles'])
    for article in all_articles['articles']:
        print('Source ID:', article['source']['id'])
        print('Source name:', article['source']['name'])
        print('Author:', article['author'])
        print('Title:', article['title'])
        print('URL:', article['url'])
        print('Publish date:', article['publishedAt'])
        print('Article text:', article['content'], '\n')

    # Create a list of tuples from the dictionary data
    source_data01 = [(a['source']['name'],
                      a['author'],
                      a['title'],
                      a['url'],
                      a['publishedAt'],
                      a['content'])
                     for a in all_articles['articles']]

    api_lst.extend(source_data01)
    #print(api_lst)
    print(len(api_lst))

## Data Collection:

### Connect to API to access URLs

#### Set API filter parameters

**Extract Training Data for Classifier**

Two Left leaning and Two Right leaning news sources are queried, collected, cleaned, and prepared for model training and test

In [ ]:
# Parameters to extract API News data from 4 sources:

source_lst = ['the-washington-post','cnn','fox-news','breitbart-news']


date_lst = ['2023-05-18', '2023-05-19', '2023-05-21', '2023-05-22', '2023-05-23'] 


q_word_lst = ['justice OR surveillance', 'healthcare OR "health care"',
              '(political AND (bias OR party)) OR republican OR democrat OR election',
              'security AND (social OR national)', '(policy AND (drug OR "affirmative action")) OR regulate OR regulation']



**Extract Client Data for Business Application of our Classifer**

Data from what is considered a Centered news source is extracted, preprocessed, and prepared to run through our trained Classifier. We report back the data gathered, the overall lean found in that data, and give details on the lean, if a political lean is found in the data.

In [ ]:
# Parameters to extract API News data from 1 centered source:

source_lst = ['associated-press']


date_lst = ['2023-05-18', '2023-05-19', '2023-05-20', '2023-05-21', '2023-05-22', '2023-05-23',
            '2023-05-24', '2023-05-25', '2023-05-17', '2023-05-16', '2023-05-15', '2023-05-14',
            '2023-05-26', '2023-05-27', '2023-05-28', '2023-05-29', '2023-05-30', '2023-05-31',
            '2023-05-13', '2023-05-12']

#, '2023-05-11', '2023-05-10', '2023-05-09', '2023-05-08']

q_word_lst = ['justice OR surveillance', 'healthcare OR "health care"',
              '(political AND (bias OR party)) OR republican OR democrat OR election',
              'security AND (social OR national)', '(policy AND (drug OR "affirmative action")) OR regulate OR regulation']



In [ ]:
# Parameters to extract API News data from 1 centered source:

source_lst = ['the-hill']


date_lst = ['2023-05-18', '2023-05-19', '2023-05-20', '2023-05-21', '2023-05-22', '2023-05-23',
            '2023-05-24', '2023-05-25', '2023-05-17', '2023-05-16', '2023-05-15', '2023-05-14',
            '2023-05-26', '2023-05-27', '2023-05-28', '2023-05-29', '2023-05-30', '2023-05-31']
#date_lst = ['2023-06-01','2023-06-02','2023-06-03','2023-06-04','2023-06-05','2023-06-06','2023-06-07',
#            '2023-06-08','2023-06-09','2023-06-10','2023-06-11','2023-06-12','2023-06-13','2023-06-14',
#            '2023-06-15']

q_word_lst = ['justice OR surveillance', 'healthcare OR "health care"',
              '(political AND (bias OR party)) OR republican OR democrat OR election',
              'security AND (social OR national)', '(policy AND (drug OR "affirmative action")) OR regulate OR regulation']



#### Access API (collect URLs)

Using the defined parameters above, extract URL and other data information from the desired news sources.

In [1]:
api_record_lst01 = []
for s in source_lst:
    print(f'Source: {s}')
    for d in date_lst:
        print(f'Date: {d}')
        for q in q_word_lst:
            print(f'Query word: {q}')
            time.sleep(5 + 11 * random.random())
            news_api_urls(q=q,
                          s=s,
                          d_from=d,
                          d_to=d,
                          api_lst=api_record_lst01)
            print(s, d, q)
    time.sleep(10 + 13 * random.random())

#print(api_record_lst01)
#print(len(api_record_lst01))

#### Save into DataFrames:

Four sources data (used to TRAIN and TEST the MODEL):

In [ ]:
api_record_df2 = pd.DataFrame (list (api_record_lst01),
                                   columns = ['Source','Author','Title','URL','date','content'])

Centered Client data (stakeholder's data that will be CLASSIFIED using above mentioned MODEL):

In [ ]:
api_record_df = pd.DataFrame (list (api_record_lst01),
                                   columns = ['Source','Author','Title','URL','date','content'])
#api_record_df.to_csv("News_API_Reuters2.csv", sep=',')
api_record_df.to_csv("News_API_TheHill3.csv", sep=',')

In [ ]:
api_record_df.shape

(69, 6)

#### Code to combine different dataframes for training (my portion): (no need to run again)

In [ ]:
api_record_df_final = pd.concat([api_record_df2, api_record_df])


In [ ]:
api_record_df_final.to_csv("News_API_FOX_CNN_Breitbert_May18_23_May31.csv", sep=',')
api_record_df.to_csv("News_API_FOX_CNN_Breitbert_May20_May31.csv", sep=',')
api_record_df2.to_csv("News_API_FOX_CNN_Breitbert_May1819_May2123.csv", sep=',')

In [ ]:
api_record_df_filtered = api_record_df[api_record_df.Source != 'The Washington Post']

In [ ]:
api_data_df3=pd.read_csv('News_API_FOX_CNN_Breitbert_May18_23_May31_2.csv')


In [ ]:
api_record_df_final3 = api_data_df3[api_data_df3.Source != 'The Washington Post']

In [ ]:
api_record_df_final3.to_csv("News_API_FOX_CNN_Breitbert_May18_23_May31_3.csv", sep=',')
api_record_df_final3.shape
api_record_df_filtered
api_record_df2.shape

In [ ]:
api_record_df_final2 = pd.concat([api_record_df2, api_record_df_filtered])
api_record_df_final2.shape

In [ ]:
api_record_df_final2.to_csv("News_API_FOX_CNN_Breitbert_May18_23_May31_2.csv", sep=',')
api_data_df_final3=pd.read_csv('News_API_FOX_CNN_Breitbert_May18_23_May31_2.csv')



### HTML scrape of URLs (to collect Article content)

#### FOX, CNN, Breitbert:

In [ ]:
slct_tbl_full_df02 = api_record_df_final3.copy()



slct_tbl_full_df02['article_parsed'] = ''

total_urls = len(slct_tbl_full_df02)

# Start timer
start_time = datetime.date.today()


for i, row in enumerate(slct_tbl_full_df02.itertuples(), 1):
    print(row[5])
    article=[]
    #url = slct_tbl_full_df02.at[i,'URL']
    url=row[5]
    response = requests.get(url)
    time.sleep(5 + 10*random.random())

    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        if script_tag == None:
            print('\nJSON Object == None: Index:', i-1, '; source:',
                  row[2],
                  '; URL:', row[5]) 
        else:
            article_json = json.loads(script_tag.string)
            article_content = article_json['articleBody']
            slct_tbl_full_df02.at[row.Index, 'article_parsed'] = article_content

    except KeyError:
        print('\nMissing JSON key: Index:', i-1, '; source:',
              row[2],
              '; URL:', row[5]) 
        article_body = soup.find('div', class_='article__content-container') 

        if article_body is None: #forfoxandbreitbert(sometimes)
            print('Class != article__content-container')
            article_body = soup.find('p', class_="speakable")
            if article_body is None: #breitbert(most)
                print('Class != speakable')
                article_body = soup.find('div', class_='entry-content')
                if article_body is None: #WashPost
                    print('Class != entry-content')
                    article_body = soup.find('div',class_='article-body')

        if article_body is not None:
            article_text = article_body.get_text()
            slct_tbl_full_df02.at[row.Index, 'article_parsed'] = article_text
        else:
            print('Could not parse')

    print('.', end='')
        
# End timer script
end_time = datetime.date.today()
time_elapse = end_time - start_time
print(f'Start Time = {start_time}')
print(f'End Time = {end_time}')
print(f'Elapsed Time = {time_elapse}')

In [ ]:
slct_tbl_full_df02.to_csv("News_API_FOX_CNN_Breitbert_May18_23_May31_3.csv", sep=',')

#### The Hill:

In [ ]:
slct_tbl_full_df02 = api_record_df.copy()


slct_tbl_full_df02['article_parsed'] = ''

total_urls = len(slct_tbl_full_df02)

# Start timer
start_time = datetime.date.today()


for i, row in enumerate(slct_tbl_full_df02.itertuples(), 1):
    print(row[4])
    article=[]
    #url = slct_tbl_full_df02.at[i,'URL']
    url=row[4]
    response = requests.get(url)
    time.sleep(5 + 10*random.random())

    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        if script_tag == None:
            print('\nJSON Object == None: Index:', i-1, '; source:',
                  row[1],
                  '; URL:', row[4]) 
        else:
            article_json = json.loads(script_tag.string)
            article_content = article_json['articleBody']
            slct_tbl_full_df02.at[row.Index, 'article_parsed'] = article_content

    except KeyError:
        print('\nMissing JSON key: Index:', i-1, '; source:',
              row[1],
              '; URL:', row[4]) 
        #article_body = soup.find('div', class_='article__content__6hMn9') #Reuters scrape attempt
        #article_body = soup.find('div', class_='Article') #AP scrape attempt
        article_body = soup.find('div', class_='article__text | body-copy | flow') #theHill

        if article_body is None: #forfoxandbreitbert(sometimes)
            print('Class != article__content-container')
            article_body = soup.find('p', class_="speakable")
            if article_body is None: #breitbert(most)
                print('Class != speakable')
                article_body = soup.find('div', class_='entry-content')
                if article_body is None: #WashPost
                    print('Class != entry-content')
                    article_body = soup.find('div',class_='article-body')

        if article_body is not None:
            article_text = article_body.get_text()
            slct_tbl_full_df02.at[row.Index, 'article_parsed'] = article_text
        else:
            print('Could not parse')

    print('.', end='')
        
# End timer script
end_time = datetime.date.today()
time_elapse = end_time - start_time
print(f'Start Time = {start_time}')
print(f'End Time = {end_time}')
print(f'Elapsed Time = {time_elapse}')

https://thehill.com/homenews/house/4010248-greene-plans-file-articles-impeachment-biden/

Missing JSON key: Index: 0 ; source: The Hill ; URL: https://thehill.com/homenews/house/4010248-greene-plans-file-articles-impeachment-biden/
.https://thehill.com/policy/energy-environment/4011061-markey-bill-would-restore-ban-on-u-s-fossil-fuel-exports/

Missing JSON key: Index: 1 ; source: The Hill ; URL: https://thehill.com/policy/energy-environment/4011061-markey-bill-would-restore-ban-on-u-s-fossil-fuel-exports/
.https://thehill.com/policy/healthcare/4009038-what-a-default-could-mean-for-medicaid-and-other-federal-benefits/

Missing JSON key: Index: 2 ; source: The Hill ; URL: https://thehill.com/policy/healthcare/4009038-what-a-default-could-mean-for-medicaid-and-other-federal-benefits/
.https://thehill.com/business/4010770-with-biden-in-japan-harris-pushes-white-house-message-on-debt-ceiling/

Missing JSON key: Index: 3 ; source: The Hill ; URL: https://thehill.com/business/4010770-with-bid

#### Check Data:

In [ ]:
slct_tbl_full_df02.isnull().sum()

Source            0
Author            0
Title             0
URL               0
date              0
content           0
article_parsed    0
dtype: int64

In [ ]:
api_record_df_latest=slct_tbl_full_df02.copy()

Make sure Word count for parsed article is correct:

In [ ]:
api_record_df_latest['word_count'] = api_record_df_latest['article_parsed'].apply(lambda x: len(str(x).split(" ")))

Only include rows with a word_count greater than 1:

In [ ]:
api_record_df_latest = api_record_df_latest[api_record_df_latest['word_count']>1]

Save Data to CSV:

In [ ]:
api_record_df_latest.to_csv("News_API_TheHill_wordcount3.csv", sep=',')
